In [32]:
import sqlite3
from sklearn import metrics
from scipy.sparse import hstack
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm 
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns

In [225]:
cnx = sqlite3.connect('DebateOrg/ddo-V1.db')
c = cnx.cursor()
#debates title with users and category
debates = pd.read_sql_query('SELECT participant_1_link,category,url,title, participant_1_position FROM debates ', cnx)
#users
users = pd.read_sql_query('SELECT * FROM users', cnx)
#selecting only the useful columns like party,religion, ethnicity etc
users=users[['url','ideology','party','gender','ethnicity','religion','win_ratio']]
interval = re.compile('^(\d+\.\d+)\%$')

def read(elem):
    m=interval.search(elem)
    if m:
        num = float(m.group(1))
        return((num)/100)
users['win_ratio']=users['win_ratio'].apply(read)
#deb_users=pd.merge(debates, users, how='left', left_on='participant_1_link', right_on='url')
#deb_users=deb_users[['category','title','url_x','url_y','ideology','party','gender','ethnicity','religion','participant_1_position','win_ratio']]
#deb_users=deb_users.rename(columns=({'url_y':'username', 'url_x':'url', 'participant_1_position':'user_position'}))
username = re.compile('^.(\w+).$')

def find_username(elem):
    m=username.search(str(elem))
    if m:
        un = m.group(1)
        return un
    
debates['username']=debates['participant_1_link'].apply(find_username)
users["url"]=users["url"].apply(find_username)

In [226]:
users=users[users["religion"]!="Not Saying"]
users=users[users["party"]!="Not Saying"]
users=users[users["ideology"]!="Not Saying"]
users=users[users["ethnicity"]!="Not Saying"]
users=users[users["gender"]!="Prefer not to say"]

In [227]:
def groupping_gender(x):
    #gender
    if x in ("Agender","Androgyne","Bigender","Genderqueer","Transgender Female","Transgender Male"):
        x = "LGBTQIAPK+" 
    return x 
def groupping_ideol(x):    
    #ideology
    if x in ("Apathetic","Undecided","Other"):
        x="Other"
    elif x in ("Anarchist","Communist","Green","Socialist"):
        x="Left"
    elif x in ("Labor","Moderate","Progressive"):
        x="Center"
    else:
        x= "Right"
    return x

def groupping_religion(x):
    if x in ("Agnostic","Atheist"):
        x=x
    elif x in ("Christian - Catholic", "Christian","Christian - Mennonite", "Christian - Amish","Christian - Episcopalian","Christian - Jehovah's Witness","Christian - Seventh-Day Adventist","Christian - Assemblies of God","Christian - Anglican","Christian - Greek Orthodox","Christian - Presbytarian","Christian - Pentecosta","Christian - Methodist","Christian - Church of Christ","Christian - Lutheran","Christian - Latter-Day Saints","Christian - Protestant","Christian - Baptist"):
        x="Christian"
    elif x in ("Muslim - Sufi","Muslim - Shiite","Islamic","Muslim - Sunni","Muslim"):
        x="Muslim"
    else:
        x="Other"
    return x

position = re.compile('^([A-Z][a-z]+)\s.(\w+).$')

def pos(elem):
    m=position.search(str(elem))
    if m:
        un = m.group(2)
        return un

In [228]:
users["religion"]=users["religion"].apply(groupping_religion)
users["gender"]=users["gender"].apply(groupping_gender)
users["ideology"]=users["ideology"].apply(groupping_ideol)


In [229]:
users=users.reset_index()
users=users[["url","ideology","party","gender","ethnicity","religion","win_ratio"]]

In [230]:
users

,url,ideology,party,gender,ethnicity,religion,win_ratio
0,Mikal,Right,Independent,Male,White,Agnostic,0.9684
1,bluesteel,Right,Republican Party,Female,White,Christian,0.9811
2,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448
3,tejretics,Right,Democratic Party,Male,East Indian,Atheist,0.9531
4,thett3,Right,Independent,Female,White,Christian,0.9133
...,...,...,...,...,...,...,...
4163,None,Left,American Nazi Party,Male,White,Christian,0.1587
4164,FanboyMctroll,Left,Pirate Party,Male,White,Atheist,0.1379
4165,jp_porwisz10,Right,Republican Party,Male,White,Christian,0.0909
4166,hd1997,Other,Democratic Party,Female,White,Other,0.1667


In [232]:
debates

,participant_1_link,category,url,title,participant_1_position,username
0,/Mikal/,Miscellaneous,/debates/Covenys-recent-behavior-would-be-a-ne...,Covenys recent behavior would be a net detrime...,Pro (for),Mikal
1,/Mikal/,Politics,/debates/Islam-is-a-civilized-religion/1/,Islam is a civilized religion,Con (against),Mikal
2,/Mikal/,Games,/debates/Gaming-is-a-sport/4/,Gaming is a sport,Pro (for),Mikal
3,/JimShady/,Music,/debates/Rap-Battle-of-Amazingness-13/1/,"Rap Battle of Amazingness, 13",Con (against),JimShady
4,/Mikal/,Miscellaneous,/debates/Racial-Religious-Profiling-Is-a-Ratio...,Racial/Religious Profiling Is a Rational Polit...,Pro (for),Mikal
...,...,...,...,...,...,...
81794,/backwardseden/,Religion,/debates/Young-Boy-Found-at-a-New-Mexico-Compo...,Young Boy Found at a New Mexico Compound Died ...,Con (against),backwardseden
81795,/Thomasmariel33/,Politics,/debates/When-governments-employ-the-term-stri...,When governments employ the term strike (as in...,Pro (for),Thomasmariel33
81796,/Thomasmariel33/,Society,/debates/Should-the-words-poor-and-homeless-be...,Should the words poor and homeless be eradicat...,Pro (for),Thomasmariel33
81797,/Thomasmariel33/,Movies,/debates/Does-the-voice-calling-Jesses-name-as...,"Does the voice, Calling Jesse's name as she sl...",Pro (for),Thomasmariel33


In [231]:
arguments=pd.read_sql_query('SELECT * FROM arguments',cnx)
arguments['user']=arguments['user'].apply(find_username)

In [235]:
arg=pd.merge(arguments,debates, left_on="debate",right_on="url", how="left")

In [236]:
arg.columns

Index(['debate', 'text', 'user', 'side', 'round_number', 'participant_1_link',
       'category', 'url', 'title', 'participant_1_position', 'username'],
      dtype='object')

In [237]:
arg_1=arg[['text', 'user','side','category', 'title']]

In [238]:
users.columns

Index(['url', 'ideology', 'party', 'gender', 'ethnicity', 'religion',
       'win_ratio'],
      dtype='object')

In [239]:
arg_2=pd.merge(arg_1, users, left_on="user",right_on="url",how="left")


In [240]:
arg_2=arg_2.dropna().reset_index()

In [241]:
arg_2["features"]=arg_2['ideology']+' '+arg_2['party']+' '+arg_2['gender']+' '+arg_2['ethnicity']+' '+arg_2['religion']

In [242]:
arg_2.columns

Index(['index', 'text', 'user', 'side', 'category', 'title', 'url', 'ideology',
       'party', 'gender', 'ethnicity', 'religion', 'win_ratio', 'features'],
      dtype='object')

In [243]:
arg_2

,index,text,user,side,category,title,url,ideology,party,gender,ethnicity,religion,win_ratio,features
0,0,Some context on the resolution. Normally I wou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
1,2,We can dismiss most everything in his last rou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
2,4,On to rebuttals1) IntroI'm not sure what he is...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
3,6,This is a debate challenge to Cassie on a foru...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
4,8,Resending this to Zaro as this was intended fo...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94713,5908627,Don't dismiss my position as a non-argument. Y...,Mikegj1077,Con,Health,Gov. funded health care,Mikegj1077,Right,Republican Party,Male,White,Christian,0.0556,Right Republican Party Male White Christian
94714,5908629,"""UK - booming economy, Nationalized Health Ser...",Mikegj1077,Con,Health,Gov. funded health care,Mikegj1077,Right,Republican Party,Male,White,Christian,0.0556,Right Republican Party Male White Christian
94715,5912374,"Communism was terrible, It made people equally...",billsands,Pro,Politics,The USSR could have survived if it has adopted...,billsands,Left,Socialist Party,Male,Other,Agnostic,0.3125,Left Socialist Party Male Other Agnostic
94716,5912376,Where did i state marx was right about everyth...,billsands,Pro,Politics,The USSR could have survived if it has adopted...,billsands,Left,Socialist Party,Male,Other,Agnostic,0.3125,Left Socialist Party Male Other Agnostic


In [244]:
arg_2=arg_2[['text', 'user', 'side', 'category', 'title', 'url', 'ideology','party', 'gender', 'ethnicity', 'religion', 'win_ratio','features']]

In [245]:
lol=arg_2[0:3000]


In [246]:
lol

,text,user,side,category,title,url,ideology,party,gender,ethnicity,religion,win_ratio,features
0,Some context on the resolution. Normally I wou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
1,We can dismiss most everything in his last rou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
2,On to rebuttals1) IntroI'm not sure what he is...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
3,This is a debate challenge to Cassie on a foru...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
4,Resending this to Zaro as this was intended fo...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684,Right Independent Male White Agnostic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,"I maintain that the bible, comprised of Hebrew...",Kleptin,Pro,Religion,The Infallibility of the Bible,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,Other Other Male Asian Agnostic
2996,That aregument is irrelevant. While I agree wi...,Kleptin,Pro,Religion,The Infallibility of the Bible,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,Other Other Male Asian Agnostic
2997,It must have been a miscommunication. Perhaps ...,Kleptin,Pro,Religion,The Infallibility of the Bible,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,Other Other Male Asian Agnostic
2998,"""God's existance can be proven, or at least is...",Kleptin,Con,Religion,God more likely than not exists,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,Other Other Male Asian Agnostic


In [141]:
#Pre processing text
# Step - a : Remove blank rows if any.
lol['features'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
lol['features'] = [entry.lower() for entry in lol['features']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
lol['features']= [word_tokenize(entry) for entry in lol['features']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(lol['features']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        #print(word,tag)
        ## Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'title_final'
    lol.loc[index,'features_final'] = str(Final_words)

<ipython-input-141-4759d9e58b28>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lol['features'].dropna(inplace=True)
<ipython-input-141-4759d9e58b28>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lol['features'] = [entry.lower() for entry in lol['features']]
<ipython-input-141-4759d9e58b28>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [142]:
lol

,text,user,side,category,title,url,ideology,party,gender,ethnicity,religion,win_ratio,features,features_final
0,Some context on the resolution. Normally I wou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn..."
1,We can dismiss most everything in his last rou...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn..."
2,On to rebuttals1) IntroI'm not sure what he is...,Mikal,Pro,Miscellaneous,Covenys recent behavior would be a net detrime...,Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn..."
3,This is a debate challenge to Cassie on a foru...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn..."
4,Resending this to Zaro as this was intended fo...,Mikal,Con,Politics,Islam is a civilized religion,Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,"I maintain that the bible, comprised of Hebrew...",Kleptin,Pro,Religion,The Infallibility of the Bible,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']"
2996,That aregument is irrelevant. While I agree wi...,Kleptin,Pro,Religion,The Infallibility of the Bible,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']"
2997,It must have been a miscommunication. Perhaps ...,Kleptin,Pro,Religion,The Infallibility of the Bible,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']"
2998,"""God's existance can be proven, or at least is...",Kleptin,Con,Religion,God more likely than not exists,Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']"


In [143]:
#Pre processing text
# Step - a : Remove blank rows if any.
lol['title'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
lol['title'] = [entry.lower() for entry in lol['title']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
lol['title']= [word_tokenize(entry) for entry in lol['title']]


<ipython-input-143-05faa225144b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lol['title'].dropna(inplace=True)
<ipython-input-143-05faa225144b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lol['title'] = [entry.lower() for entry in lol['title']]
<ipython-input-143-05faa225144b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [144]:
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(lol['title']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        #print(word,tag)
        ## Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'title_final'
    lol.loc[index,'title_final'] = str(Final_words)

C:\Users\Lilia\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Lilia\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [145]:
lol

,text,user,side,category,title,url,ideology,party,gender,ethnicity,religion,win_ratio,features,features_final,title_final
0,Some context on the resolution. Normally I wou...,Mikal,Pro,Miscellaneous,"[covenys, recent, behavior, would, be, a, net,...",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['covenys', 'recent', 'behavior', 'would', 'ne..."
1,We can dismiss most everything in his last rou...,Mikal,Pro,Miscellaneous,"[covenys, recent, behavior, would, be, a, net,...",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['covenys', 'recent', 'behavior', 'would', 'ne..."
2,On to rebuttals1) IntroI'm not sure what he is...,Mikal,Pro,Miscellaneous,"[covenys, recent, behavior, would, be, a, net,...",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['covenys', 'recent', 'behavior', 'would', 'ne..."
3,This is a debate challenge to Cassie on a foru...,Mikal,Con,Politics,"[islam, is, a, civilized, religion]",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['islam', 'civilized', 'religion']"
4,Resending this to Zaro as this was intended fo...,Mikal,Con,Politics,"[islam, is, a, civilized, religion]",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['islam', 'civilized', 'religion']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,"I maintain that the bible, comprised of Hebrew...",Kleptin,Pro,Religion,"[the, infallibility, of, the, bible]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['infallibility', 'bible']"
2996,That aregument is irrelevant. While I agree wi...,Kleptin,Pro,Religion,"[the, infallibility, of, the, bible]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['infallibility', 'bible']"
2997,It must have been a miscommunication. Perhaps ...,Kleptin,Pro,Religion,"[the, infallibility, of, the, bible]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['infallibility', 'bible']"
2998,"""God's existance can be proven, or at least is...",Kleptin,Con,Religion,"[god, more, likely, than, not, exists]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['god', 'likely', 'exists']"


In [146]:
# Step - a : Remove blank rows if any.
lol['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
lol['text'] = [entry.lower() for entry in lol['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
lol['text']= [word_tokenize(entry) for entry in lol['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(lol['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'title_final'
    lol.loc[index,'text_final'] = str(Final_words)

<ipython-input-146-75974beb430e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lol['text'].dropna(inplace=True)
<ipython-input-146-75974beb430e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lol['text'] = [entry.lower() for entry in lol['text']]
<ipython-input-146-75974beb430e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [147]:
lol

,text,user,side,category,title,url,ideology,party,gender,ethnicity,religion,win_ratio,features,features_final,title_final,text_final
0,"[some, context, on, the, resolution, ., normal...",Mikal,Pro,Miscellaneous,"[covenys, recent, behavior, would, be, a, net,...",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['covenys', 'recent', 'behavior', 'would', 'ne...","['context', 'resolution', 'normally', 'would',..."
1,"[we, can, dismiss, most, everything, in, his, ...",Mikal,Pro,Miscellaneous,"[covenys, recent, behavior, would, be, a, net,...",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['covenys', 'recent', 'behavior', 'would', 'ne...","['dismiss', 'everything', 'last', 'round', 'pe..."
2,"[on, to, rebuttals1, ), introi, 'm, not, sure,...",Mikal,Pro,Miscellaneous,"[covenys, recent, behavior, would, be, a, net,...",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['covenys', 'recent', 'behavior', 'would', 'ne...","['introi', 'sure', 'say', 'start', 'last', 'ro..."
3,"[this, is, a, debate, challenge, to, cassie, o...",Mikal,Con,Politics,"[islam, is, a, civilized, religion]",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['islam', 'civilized', 'religion']","['debate', 'challenge', 'cassie', 'forum', 'po..."
4,"[resending, this, to, zaro, as, this, was, int...",Mikal,Con,Politics,"[islam, is, a, civilized, religion]",Mikal,Right,Independent,Male,White,Agnostic,0.9684,"[right, independent, male, white, agnostic]","['right', 'independent', 'male', 'white', 'agn...","['islam', 'civilized', 'religion']","['resending', 'zaro', 'intend', 'adversary', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,"[i, maintain, that, the, bible, ,, comprised, ...",Kleptin,Pro,Religion,"[the, infallibility, of, the, bible]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['infallibility', 'bible']","['maintain', 'bible', 'comprise', 'hebrew', 's..."
2996,"[that, aregument, is, irrelevant, ., while, i,...",Kleptin,Pro,Religion,"[the, infallibility, of, the, bible]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['infallibility', 'bible']","['aregument', 'irrelevant', 'agree', 'relate',..."
2997,"[it, must, have, been, a, miscommunication, .,...",Kleptin,Pro,Religion,"[the, infallibility, of, the, bible]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['infallibility', 'bible']","['must', 'miscommunication', 'perhaps', 'clear..."
2998,"[``, god, 's, existance, can, be, proven, ,, o...",Kleptin,Con,Religion,"[god, more, likely, than, not, exists]",Kleptin,Other,Other,Male,Asian,Agnostic,0.9448,"[other, other, male, asian, agnostic]","['male', 'asian', 'agnostic']","['god', 'likely', 'exists']","['god', 'existance', 'prove', 'least', 'reason..."


In [151]:
lol.to_csv("tokenized.csv")

In [152]:
np.random.seed(500)

In [153]:
lol=lol.dropna()

In [154]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(lol[["title_final",'text_final',"side"]],lol['religion'],test_size=0.3)

In [155]:
Encoder = LabelEncoder()
Train_Y_1 = Encoder.fit_transform(Train_Y)
Test_Y_1 = Encoder.fit_transform(Test_Y)


In [156]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["title_final"])
Train_X_Tfidf_title = Tfidf_vect.transform(Train_X["title_final"])
Test_X_Tfidf_title = Tfidf_vect.transform(Test_X["title_final"])


In [157]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["text_final"])
Train_X_Tfidf_text = Tfidf_vect.transform(Train_X["text_final"])
Test_X_Tfidf_text = Tfidf_vect.transform(Test_X["text_final"])


In [158]:
Train_X_1_pos = Encoder.fit_transform(Train_X['side'])
Test_X_1_pos = Encoder.fit_transform(Test_X['side'])


In [159]:
mat_train= hstack([Train_X_Tfidf_title,Train_X_Tfidf_text])
X_train_tfidf = hstack([mat_train, Train_X_1_pos.reshape(-1, 1)])

In [160]:
mat_test= hstack([Test_X_Tfidf_title,Test_X_Tfidf_text])
X_test_tfidf = hstack([mat_test, Test_X_1_pos.reshape(-1, 1)])

In [161]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,Train_Y_1)

SVC(gamma='auto', kernel='linear')

In [162]:
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y_1)*100)

SVM Accuracy Score ->  87.0


In [163]:
print(metrics.classification_report(Test_Y_1, predictions_SVM,target_names=["Agnostic","Atheist","Christian","Muslim","Other"]))

              precision    recall  f1-score   support

    Agnostic       0.89      0.94      0.91       462
     Atheist       0.81      0.67      0.73        81
   Christian       0.87      0.88      0.88       294
      Muslim       0.71      1.00      0.83         5
       Other       0.82      0.53      0.65        58

    accuracy                           0.87       900
   macro avg       0.82      0.80      0.80       900
weighted avg       0.87      0.87      0.87       900



In [195]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(lol[["title_final",'text_final',"side"]],lol['ideology'],test_size=0.3)
Encoder = LabelEncoder()
Train_Y_1 = Encoder.fit_transform(Train_Y)
Test_Y_1 = Encoder.fit_transform(Test_Y)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["title_final"])
Train_X_Tfidf_title = Tfidf_vect.transform(Train_X["title_final"])
Test_X_Tfidf_title = Tfidf_vect.transform(Test_X["title_final"])
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["text_final"])
Train_X_Tfidf_text = Tfidf_vect.transform(Train_X["text_final"])
Test_X_Tfidf_text = Tfidf_vect.transform(Test_X["text_final"])
Train_X_1_pos = Encoder.fit_transform(Train_X['side'])
Test_X_1_pos = Encoder.fit_transform(Test_X['side'])
mat_train= hstack([Train_X_Tfidf_title,Train_X_Tfidf_text])
X_train_tfidf = hstack([mat_train, Train_X_1_pos.reshape(-1, 1)])
mat_test= hstack([Test_X_Tfidf_title,Test_X_Tfidf_text])
X_test_tfidf = hstack([mat_test, Test_X_1_pos.reshape(-1, 1)])

In [196]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,Train_Y_1)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y_1)*100)

SVM Accuracy Score ->  89.0


In [201]:
print(metrics.classification_report(Test_Y_1, predictions_SVM,target_names=["Center","Left","Center","Right"]))

              precision    recall  f1-score   support

      Center       0.74      0.57      0.64        46
        Left       0.91      0.50      0.65        40
      Center       0.88      0.79      0.83       184
       Right       0.90      0.97      0.93       630

    accuracy                           0.89       900
   macro avg       0.86      0.71      0.76       900
weighted avg       0.89      0.89      0.88       900



In [202]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(lol[["title_final",'text_final',"side"]],lol['gender'],test_size=0.3)
Encoder = LabelEncoder()
Train_Y_1 = Encoder.fit_transform(Train_Y)
Test_Y_1 = Encoder.fit_transform(Test_Y)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["title_final"])
Train_X_Tfidf_title = Tfidf_vect.transform(Train_X["title_final"])
Test_X_Tfidf_title = Tfidf_vect.transform(Test_X["title_final"])
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["text_final"])
Train_X_Tfidf_text = Tfidf_vect.transform(Train_X["text_final"])
Test_X_Tfidf_text = Tfidf_vect.transform(Test_X["text_final"])
Train_X_1_pos = Encoder.fit_transform(Train_X['side'])
Test_X_1_pos = Encoder.fit_transform(Test_X['side'])
mat_train= hstack([Train_X_Tfidf_title,Train_X_Tfidf_text])
X_train_tfidf = hstack([mat_train, Train_X_1_pos.reshape(-1, 1)])
mat_test= hstack([Test_X_Tfidf_title,Test_X_Tfidf_text])
X_test_tfidf = hstack([mat_test, Test_X_1_pos.reshape(-1, 1)])

In [203]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,Train_Y_1)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y_1)*100)

SVM Accuracy Score ->  94.0


In [207]:
print(metrics.classification_report(Test_Y_1, predictions_SVM,target_names=["Female","LGBTQIAPK+","Male"]))

              precision    recall  f1-score   support

      Female       0.86      0.88      0.87       192
  LGBTQIAPK+       0.00      0.00      0.00         3
        Male       0.96      0.96      0.96       705

    accuracy                           0.94       900
   macro avg       0.61      0.61      0.61       900
weighted avg       0.94      0.94      0.94       900



C:\Users\Lilia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lilia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lilia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [208]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(lol[["title_final",'text_final',"side"]],lol['ethnicity'],test_size=0.3)
Encoder = LabelEncoder()
Train_Y_1 = Encoder.fit_transform(Train_Y)
Test_Y_1 = Encoder.fit_transform(Test_Y)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["title_final"])
Train_X_Tfidf_title = Tfidf_vect.transform(Train_X["title_final"])
Test_X_Tfidf_title = Tfidf_vect.transform(Test_X["title_final"])
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(lol["text_final"])
Train_X_Tfidf_text = Tfidf_vect.transform(Train_X["text_final"])
Test_X_Tfidf_text = Tfidf_vect.transform(Test_X["text_final"])
Train_X_1_pos = Encoder.fit_transform(Train_X['side'])
Test_X_1_pos = Encoder.fit_transform(Test_X['side'])
mat_train= hstack([Train_X_Tfidf_title,Train_X_Tfidf_text])
X_train_tfidf = hstack([mat_train, Train_X_1_pos.reshape(-1, 1)])
mat_test= hstack([Test_X_Tfidf_title,Test_X_Tfidf_text])
X_test_tfidf = hstack([mat_test, Test_X_1_pos.reshape(-1, 1)])

In [209]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,Train_Y_1)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y_1)*100)

SVM Accuracy Score ->  90.22222222222223


In [223]:
print(metrics.classification_report(Test_Y_1, predictions_SVM,target_names=['Asian','Black','Latino','Middle Eastern','Native American','Other','Pacific Islander','White']))

                  precision    recall  f1-score   support

           Asian       0.81      0.71      0.76       130
           Black       0.92      0.71      0.80        17
          Latino       1.00      0.27      0.43        11
  Middle Eastern       0.50      1.00      0.67         1
 Native American       1.00      1.00      1.00         4
           Other       0.44      0.27      0.33        15
Pacific Islander       1.00      0.67      0.80         3
           White       0.92      0.97      0.94       719

        accuracy                           0.90       900
       macro avg       0.83      0.70      0.72       900
    weighted avg       0.90      0.90      0.90       900

